In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # keepign CIFAR-100 native size
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [6]:
# loading pretrained model ResNet18
model = models.resnet18(pretrained=True)

# modifying first conv layer for 32x32 (reduce stride to keep more spatial info)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # remove the initial maxpool for small images

# replacing final layer for CIFAR-100
model.fc = nn.Linear(model.fc.in_features, 100)

model = model.to(device)

# loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

/home/BTECH_7TH_SEM/Documents/MML/MMLenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/BTECH_7TH_SEM/Documents/MML/MMLenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
23.5%

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/BTECH_7TH_SEM/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [7]:
# training function
def train(epoch):
    model.train()
    correct, total, running_loss = 0, 0, 0.0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f"Epoch {epoch}: Train Loss: {running_loss/len(trainloader):.3f}, "
          f"Accuracy: {100.*correct/total:.2f}%")

In [8]:
# test function
def test():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return acc


In [9]:
best_acc = 0
for epoch in range(1, 21):
    train(epoch)
    acc = test()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_cifar100_resnet18.pth")
        print("Model saved!")

print(f"\nBest Test Accuracy Achieved: {best_acc:.2f}%")

Epoch 1: Train Loss: 2.213, Accuracy: 42.66%
Test Accuracy: 58.16%
Model saved!
Epoch 2: Train Loss: 1.045, Accuracy: 69.11%
Test Accuracy: 65.12%
Model saved!
Epoch 3: Train Loss: 0.602, Accuracy: 81.68%
Test Accuracy: 67.53%
Model saved!
Epoch 4: Train Loss: 0.309, Accuracy: 90.89%
Test Accuracy: 70.19%
Model saved!
Epoch 5: Train Loss: 0.127, Accuracy: 96.89%
Test Accuracy: 70.40%
Model saved!
Epoch 6: Train Loss: 0.046, Accuracy: 99.25%
Test Accuracy: 72.95%
Model saved!
Epoch 7: Train Loss: 0.018, Accuracy: 99.84%
Test Accuracy: 73.88%
Model saved!
Epoch 8: Train Loss: 0.010, Accuracy: 99.93%
Test Accuracy: 74.10%
Model saved!
Epoch 9: Train Loss: 0.007, Accuracy: 99.94%
Test Accuracy: 74.15%
Model saved!
Epoch 10: Train Loss: 0.006, Accuracy: 99.96%
Test Accuracy: 74.42%
Model saved!
Epoch 11: Train Loss: 0.006, Accuracy: 99.96%
Test Accuracy: 74.35%
Epoch 12: Train Loss: 0.005, Accuracy: 99.96%
Test Accuracy: 74.19%
Epoch 13: Train Loss: 0.005, Accuracy: 99.96%
Test Accuracy: 74